## First, let's install and import all required libraries

In [ ]:
#!pip install -r requirements.txt

In [ ]:
import collections                        #This library adds some extras to the standard python data structures
import folium                             #Great library for plotting data on maps
import json                               #This library allows us to load and hyndle json files with python
import os                                 #This module is usefull for handling paths and directories
import webbrowser                         #This library allows to handle the webbrowser from python
import time
import datetime
import pandas as pd
from folium.plugins import HeatMap
from progressbar import ProgressBar, Bar, ETA, Percentage

## Now, let's load and analyze our location history json file 

In [ ]:
data = json.load(open('Location_History_Juan.json'))

In [ ]:
data

In [ ]:
type(data)

In [ ]:
data["locations"]

In [ ]:
df = pd.DataFrame(data["locations"])
df

In [ ]:
df = df.drop(columns=['activity'])
df

## Now we can extract and tranform the data we need to make our map 

In [ ]:
coordinates = collections.defaultdict(int)

In [ ]:
coordinates

In [ ]:
max_coordinates = (0, 0)

In [ ]:
max_magnitude = 0

In [ ]:
"""Here you transfor the coordinates given by google in actual longitude and latitude coordinates and
show the progress using the progress bar widget"""

w = [Bar(), Percentage(), " ", ETA()]
with ProgressBar(max_value=len(data["locations"]), widgets=w) as pb:
    for i, loc in enumerate(data["locations"]):
        # print(i)
        # print(loc)
        if "latitudeE7" not in loc or "longitudeE7" not in loc:
            continue
        coords = (round(loc["latitudeE7"] / 1e7, 6),
                    round(loc["longitudeE7"] / 1e7, 6))
        # print(coords)

        
        """Here you calculate the magnitude for all coordinates"""
        
        #print(loc["timestampMs"])
        coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
        #print(coordinates[coords])
            
        if coordinates[coords] > max_magnitude:
            max_coordinates = coords
            max_magnitude = coordinates[coords]
            #print(max_coordinates)
            #print(max_magnitude)
        
        """This command updates the progress bar"""
        pb.update(i)

In [ ]:
coordinates

In [ ]:
tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

tiles = tilesoptions[0]
zoom_start = 10
radius = 7
blur = 4
min_opacity = 0.2
max_zoom = 10


In [ ]:
map_data = [(coords[0], coords[1], magnitude)
            for coords, magnitude in coordinates.items()] #coords are keys in the dict. magnitudes are the values

In [ ]:
map_data

In [ ]:
# Generate background map
m = folium.Map(location=max_coordinates,
                zoom_start=zoom_start,
                tiles=tiles)

# Generate heat map
heatmap = HeatMap(map_data,
                    max_val=max_magnitude,
                    min_opacity=min_opacity,
                    radius=radius,
                    blur=blur,
                    max_zoom=max_zoom)

m.add_child(heatmap)

## We can now save our map as an html file and launch it in the browser 

In [ ]:
output_file = tiles + 'heatmapJuan.html'

In [ ]:
m.save(output_file)

In [ ]:
webbrowser.open("file://" + os.path.realpath(output_file))

## Now let's wrap everything into a function and play with it

In [ ]:
def plotmymaps(Tiles, Zoom_start=10, Radius=7, Blur=4, Min_opacity=0.2, Max_zoom=10):

    tilesoptions = ["openstreetmap", "StamenTerrain", "stamentoner",
                    "stamenwatercolor", "cartodbpositron", "cartodbdark_matter"]

    tiles = tilesoptions[Tiles]
    zoom_start = Zoom_start
    radius = Radius
    blur = Blur
    min_opacity = Min_opacity
    max_zoom = Max_zoom

    map_data = [(coords[0], coords[1], magnitude)
                for coords, magnitude in coordinates.items()] 
                #coords are keys in the dict. magnitudes are the values

        # Generate background map
    m = folium.Map(location=max_coordinates,
                    zoom_start=zoom_start,
                    tiles=tiles)

    # Generate heat map
    heatmap = HeatMap(map_data,
                        max_val=max_magnitude,
                        min_opacity=min_opacity,
                        radius=radius,
                        blur=blur,
                        max_zoom=max_zoom)

    m.add_child(heatmap)
    output_file = tiles + 'heatmapJuan.html'
    m.save(output_file)
    webbrowser.open("file://" + os.path.realpath(output_file))

In [ ]:
plotmymaps(5, 2, 2, 2, 2, 2)

## What if we only want to plot data for a specific time range? 

In [ ]:
df

In [ ]:
min_date='2013-7-1'
max_date='2013-7-31'

In [ ]:
def transformdate(date):
    element = datetime.datetime.strptime(date,"%Y-%m-%d")
    elementtuple = element.timetuple()
    timestamp = time.mktime(elementtuple)
    return timestamp

In [ ]:
min_timestamp=(transformdate(min_date))*1000

In [ ]:
min_timestamp

In [ ]:
max_timestamp=(transformdate(max_date))*1000

In [ ]:
max_timestamp

In [ ]:
df["timestampMs"] = pd.to_numeric(df["timestampMs"])

In [ ]:
df2 = df[df.timestampMs > min_timestamp]
df2 = df2[df.timestampMs < max_timestamp]

In [ ]:
df2

In [ ]:
df2.to_json('temp.json', orient='records', lines=True)

In [ ]:
data2 = [json.loads(line) for line in open('temp.json', 'r')]

In [ ]:
data2

In [ ]:
coordinates = collections.defaultdict(int)
max_coordinates = (0, 0)
max_magnitude = 0

In [ ]:
"""Here you transfor the coordinates given by google in actual longitude and latitude coordinates and
show the progress using the progress bar widget"""

w = [Bar(), Percentage(), " ", ETA()]
with ProgressBar(max_value=len(data2), widgets=w) as pb:
    for i, loc in enumerate(data2):
        # print(i)
        # print(loc)
        if "latitudeE7" not in loc or "longitudeE7" not in loc:
            continue
        coords = (round(loc["latitudeE7"] / 1e7, 6),
                    round(loc["longitudeE7"] / 1e7, 6))
        # print(coords)

        
        """Here you calculate the magnitude for all coordinates"""
        
        #print(loc["timestampMs"])
        coordinates[coords] += 1 #these are the magnitude values we will need for the coordinates dictionary
        #print(coordinates[coords])
            
        if coordinates[coords] > max_magnitude:
            max_coordinates = coords
            max_magnitude = coordinates[coords]
            #print(max_coordinates)
            #print(max_magnitude)
        
        """This command updates the progress bar"""
        pb.update(i)

In [ ]:
plotmymaps(1)